# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [51]:
# Write your code below.

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Verify that PRICE_DATA is loaded
price_data_path = os.getenv('PRICE_DATA')
print(f"PRICE_DATA path: {price_data_path}")

PRICE_DATA path: ../../05_src/data/prices/


In [52]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [53]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(price_data_path, "**", "*.parquet"), recursive=True)

# Display the list of found Parquet files
print(f"Found {len(parquet_files)} parquet files.")
parquet_files[:5]  # show a sample of the paths

Found 2870 parquet files.


['../../05_src/data/prices/BKTI/BKTI_2012/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2012/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.0.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2015/part.1.parquet',
 '../../05_src/data/prices/BKTI/BKTI_2014/part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [54]:
#rename to remove spaces in the names
ddf.columns = ddf.columns.str.replace(" ", "_")
ddf = ddf.reset_index(drop=True)
print(ddf.columns)


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume', 'source',
       'ticker', 'Year'],
      dtype='object')


In [55]:
# Write your code below.

# Read all Parquet files into a Dask DataFrame
ddf = dd.read_parquet(parquet_files, engine='pyarrow', index=False)

# Define feature engineering function
def add_features(df):
    df = df.sort_values(['ticker', 'Date']) # Ensure sorting by Date for proper lag computation
    # Create lag features
    df['Close_lag_1'] = df.groupby('ticker')['Close'].shift(1)
    df['Adj_Close_lag_1'] = df.groupby('ticker')['Adj Close'].shift(1)
    # Create returns: (Close / Close_lag_1) - 1
    df['returns'] = (df['Close'] / df['Close_lag_1']) - 1
    # Create hi_lo_range: High - Low
    df['hi_lo_range'] = df['High'] - df['Low']
    return df

# Apply the transformation partition-wise
dd_feat = ddf.map_partitions(add_features)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [56]:
# Write your code below.

df_feat = dd_feat.compute()

df_feat['returns_ma_10'] = df_feat.groupby(
 			'ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)

# Show the first few rows
print(df_feat[['ticker', 'returns', 'returns_ma_10']].head(15))

   ticker   returns  returns_ma_10
0       A       NaN            NaN
1       A -0.082386            NaN
2       A  0.089783            NaN
3       A -0.090909            NaN
4       A  0.026563            NaN
5       A  0.003044            NaN
6       A  0.022762            NaN
7       A  0.001484            NaN
8       A  0.017778            NaN
9       A  0.027656            NaN
10      A  0.008499       0.002427
11      A  0.028090       0.013475
12      A -0.010929       0.003404
13      A -0.001381       0.012356
14      A  0.013831       0.011083


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

- No, it was not strictly necessary. Dask’s rolling operations require the data to be properly partitioned and sorted, and they may not work correctly across partition boundaries without additional configuration.
- For small to medium-sized datasets, using pandas is simpler and often more practical. It offers easier syntax and immediate feedback.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.